In [1]:
import pandas as pd

In [2]:
import pywikibot as pb
site = pb.Site("wikidata", "wikidata")
repo = site.data_repository()

In [3]:
from bs4 import BeautifulSoup
import requests
import dateparser

In [4]:
!pip install xlrd

  Using cached https://files.pythonhosted.org/packages/07/e6/e95c4eec6221bfd8528bcc4ea252a850bffcc4be88ebc367e23a1a84b0bb/xlrd-1.1.0-py2.py3-none-any.whl


In [5]:
!pip install XlsxWriter

  Using cached https://files.pythonhosted.org/packages/52/c6/56228f8bece3e6d707c7ba3ffde839a8989ccbc07ca3c2021d1b8243ee2c/XlsxWriter-1.0.4-py2.py3-none-any.whl


In [ ]:
def create_item(site, label_dict):
    new_item = pb.ItemPage(site)
    new_item.editLabels(labels=label_dict, summary="Creating Red List item")
    return new_item

In [21]:
def set_claim(item, property, string_data):
    clm = new_item.get()['claims'][property][0]
    clm_trgt = clm.getTarget()
    print(clm_trgt)
    if clm_trgt != string_data :
        claim = pb.Claim(repo, property)
        claim.setTarget(string_data)
        item.addClaim(claim, bot=False, summary="Test")
        return claim
    else :
        return None

In [14]:
red_list_df = pd.read_excel("red2.xlsx")

In [15]:
red_list_df.columns

Index(['Número', 'Nombre', 'Artículo en español', 'Wikidata ID', 'Latitud',
       'Longitud', 'Localidad', 'Localidad ID', 'Municipio', 'Municipio ID',
       'Provincia', 'Comunidad autónoma', 'Tipo', 'Tipo 2', 'Fecha inclusión',
       'Fecha salida', 'Descripción', 'URL', 'Estatus', 'Categoría', 'Imagen'],
      dtype='object')

In [23]:
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    if pd.notnull(row["Wikidata ID"]) :
        print ("--------------")
        item = pb.ItemPage(repo, row["Wikidata ID"])
        item.get()
        commons_cat = None
        if item.sitelinks and "commonswiki" in item.sitelinks and "Category:" in item.sitelinks["commonswiki"]:
            print (item.sitelinks["commonswiki"])
            commons_cat = item.sitelinks["commonswiki"].split(':')[1].strip()
            
        if item.claims and 'P373' in item.claims: # instance of
            print(item.claims['P373'][0].getTarget())
            commons_cat = item.claims['P373'][0].getTarget()

        if commons_cat != None :
            row["Categoria"] = commons
    output_df = output_df.append(row, ignore_index=True)        

--------------
Acueducto de Albatana
--------------
--------------
Category:Alcázar de Toro
Alcázar de Toro
--------------
--------------
--------------
--------------
Alqueria del Moro
--------------
Alqueria del Pi d'Alfafar
--------------
Category:Palacio de Chiloeches, Santoña
Palacio de Chiloeches, Santoña
--------------
Category:Church of San Miguel, Sasamón
Church of San Miguel, Sasamón
--------------
--------------
--------------
Category:Tower of Valmojado
Tower of Valmojado
--------------
Category:Bahía de Santander
Bahía de Santander
--------------
--------------
Category:Casa de la Sirena
Casa de la Sirena
--------------
Category:Batería de Cenizas
Batería de Cenizas
--------------
--------------
--------------
Castell de Xelva
--------------
Category:Capilla de San José, Sevilla
Capilla de San José (Sevilla)
--------------
Cartuja de Nuestra Señora de las Fuentes
--------------
--------------
Cartuja de Vall de Cristo
--------------
--------------
--------------
----------

--------------
Hermitage of Gañarul
--------------
Category:Ermita de San Ambrosio (Barbate)
Ermita de San Ambrosio (Barbate)
--------------
Category:Castro Peña del Hombre (Paradela de Muces)
Castro Peña del Hombre (Paradela de Muces)
--------------
Category:Church of San Pedro, Becerril de Campos
Church of San Pedro, Becerril de Campos
--------------
--------------
Category:Hermitage of San Juan de Socueva
Hermitage of San Juan de Socueva
--------------
Category:Iglesia de San Juan Bautista (Acín)
Iglesia de San Juan Bautista (Acín)
--------------
Category:San Pedro de la Mata, Casalgordo
San Pedro de la Mata, Casalgordo
--------------
Category:Hermitage of San Pelayo, Perazancas de Ojeda
Hermitage of San Pelayo, Perazancas de Ojeda
--------------
--------------
Category:Ermita de Santa Ana, El Toboso
Ermita de Santa Ana, El Toboso
--------------
--------------
--------------
Ermita del Calvari de Canet lo Roig
--------------
Category:Ermita del Santo Cristo (Talaván)
Ermita del Sant

--------------
Category:Palace of Valsaín
Palace of Valsaín
--------------
--------------
--------------
Category:Conde Luna Palace
Conde Luna Palace
--------------
Category:Palacio del Infantado o del Condestable, Colindres
Palacio del Infantado o del Condestable, Colindres
--------------
Marqués de Revilla's Palace, Navares de las Cuevas
--------------
Guenduláin
--------------
Category:Mosteiro de San Paio de Diomondi
Mosteiro de San Paio de Diomondi
--------------
Category:Palacio de los Zúñiga (Curiel de Duero)
Palacio de los Zúñiga (Curiel de Duero)
--------------
--------------
Doñana National Park
--------------
Category:Peñalcazar
Peñalcazar
--------------
Mosteiro de San Xulián de Moraime
--------------
--------------
Old town hall of Laredo
--------------
Category:Alcántara Bridge
Alcántara Bridge
--------------
--------------
Category:Puente de Cantillana
Puente de Cantillana
--------------
Medina bridge, Arévalo
--------------
--------------
Toro Bridge
--------------
Moch

In [44]:
top_categories = ['Patrimonio industrial', 
                  'Patrimonio Religioso', 
                  'Patrimonio Militar', 
                  'Patrimonio Civil', 
                  'Patrimonio Natural', 
                  'Patrimonio Arqueológico']
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    print ("-----------------------------")
    url = row['URL']
    text = requests.get(url).text
    table = BeautifulSoup(text, 'html.parser').find("div", {"class": "post-meta"})
    typology = [i.text for i in table.find_all('li') if "Tipología" in i.text][0]
    categories = [i.strip() for i in typology.replace("Tipología: ", "").split(',')]
    type1_categories = [cat for cat in categories if cat in top_categories and cat != 'Retirados de la lista']
    type2_categories = [cat for cat in categories if cat not in top_categories and cat != 'Retirados de la lista']
    row['Tipo 2'] = '|'.join(type2_categories)
    row['Tipo'] = '|'.join(type1_categories)
    
    start = [i.text for i in table.find_all('li') if "inclusión" in i.text][0]
    start_date = dateparser.parse(start.replace("Fecha de inclusión: ", "").replace(", ", " de "))
    print (start_date.strftime("%Y-%m-%d"))
    row["Fecha inclusión"] = start_date.strftime("%Y-%m-%d")

    try:
        end = [i.text for i in table.find_all('li') if "retirada" in i.text][0]
        end_date = dateparser.parse(end.replace("Fecha de retirada: ", "").replace(", ", " de "))
        print (end_date.strftime("%Y-%m-%d"))
        row["Fecha salida"] = end_date.strftime("%Y-%m-%d")
    except :
        pass

    output_df = output_df.append(row, ignore_index=True)

-----------------------------
2016-06-24
-----------------------------
2008-01-28
-----------------------------
2014-09-21
2016-05-30
-----------------------------
2010-01-18
-----------------------------
2014-12-26
2015-02-21
-----------------------------
2018-04-26
-----------------------------
2010-08-17
-----------------------------
2010-08-17
-----------------------------
2014-11-16
-----------------------------
2008-01-04
-----------------------------
2015-10-21
-----------------------------
2015-07-30
-----------------------------
2014-11-16
-----------------------------
2008-01-29
-----------------------------
2015-11-04
-----------------------------
2008-10-03
2016-10-14
-----------------------------
2014-03-14
-----------------------------
2007-12-17
2012-08-08
-----------------------------
2008-09-29
-----------------------------
2009-07-06
-----------------------------
2008-01-14
-----------------------------
2014-05-28
-----------------------------
2010-08-16
-------------

2014-10-19
-----------------------------
2013-12-20
-----------------------------
2015-02-05
-----------------------------
2010-02-09
2015-11-25
-----------------------------
2007-12-17
-----------------------------
2014-05-02
-----------------------------
2007-12-02
-----------------------------
2014-02-21
-----------------------------
2007-12-02
2015-10-20
-----------------------------
2008-02-20
-----------------------------
2014-03-17
-----------------------------
2009-01-12
-----------------------------
2014-02-07
-----------------------------
2015-03-12
-----------------------------
2014-02-13
-----------------------------
2010-02-09
-----------------------------
2013-11-01
-----------------------------
2013-08-19
-----------------------------
2015-05-12
-----------------------------
2013-01-07
-----------------------------
2014-03-14
-----------------------------
2010-09-10
-----------------------------
2014-04-18
-----------------------------
2007-11-21
2017-10-24
-------------

2011-07-03
-----------------------------
2008-02-14
-----------------------------
2015-06-15
-----------------------------
2008-06-17
-----------------------------
2013-07-18
-----------------------------
2007-12-16
2016-05-03
-----------------------------
2013-03-22
-----------------------------
2010-09-12
-----------------------------
2008-10-16
-----------------------------
2014-05-21
-----------------------------
2008-01-28
-----------------------------
2008-03-27
-----------------------------
2012-03-27
-----------------------------
2009-07-14
-----------------------------
2012-10-25
-----------------------------
2008-07-29
2009-09-17
-----------------------------
2010-03-03
2011-04-11
-----------------------------
2007-12-02
-----------------------------
2008-10-02
-----------------------------
2011-02-10
2014-08-17
-----------------------------
2015-04-14
-----------------------------
2014-05-02
2015-06-02
-----------------------------
2008-11-24
2012-12-05
---------------------

2011-01-14
-----------------------------
2007-11-13
2014-08-26
-----------------------------
2014-03-04
-----------------------------
2013-03-14
-----------------------------
2014-12-18
-----------------------------
2011-01-15
-----------------------------
2008-02-20
-----------------------------
2009-06-01
-----------------------------
2010-02-04
-----------------------------
2008-02-14
-----------------------------
2008-01-21
-----------------------------
2008-02-20
-----------------------------
2014-11-01
-----------------------------
2008-06-17
-----------------------------
2009-06-21
-----------------------------
2008-02-25
-----------------------------
2013-07-14
-----------------------------
2014-11-30
-----------------------------
2014-09-19
-----------------------------
2012-07-12
-----------------------------
2013-10-25
2018-04-26
-----------------------------
2008-10-02
-----------------------------
2007-12-02
-----------------------------
2016-02-15
------------------------

In [41]:
writer = pd.ExcelWriter('simple.xlsx', engine='xlsxwriter')
output_df.to_excel(writer, sheet_name='Sheet1')
writer.save()